# Demonstration of the Entmoot API

This notebook includes comparisons of the Entmoot strategy to other Bofire 
strategies.

## Defining model parameters

Model parameters to be passed into `EntingStrategy` can be defined in a few ways. 
Throughout this notebook, they are defined using a dictionary of parameters. 
However, all of the below are valid. 
The use of `EntingParams` is encouraged, however it does require more imports to use.

In [1]:
from entmoot.models.model_params import EntingParams, UncParams, TreeTrainParams, TrainParams

# all of these are valid to pass into `EntingStrategy(enting_params=params)`

# as a dictionary
params0 = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
# populate EntingParams with dictionary unpacking
params1 = EntingParams(**params0)
# use objects directly
params2 = EntingParams(
    unc_params=UncParams(
        dist_metric="l1",
        acq_sense="exploration"
    )
)
# explictly create all dataclass instances
params3 = EntingParams(
    unc_params=UncParams(
        dist_metric="l1",
        acq_sense="exploration"
    ),
    tree_train_params=TreeTrainParams(
        train_params=TrainParams()
    )
)

params1 == params2 == params3

True

## Single objective Bayesian Optimization strategy

This section includes a comparison to the Sobo strategy given in the "Getting 
Started" docs. The API is identical, with additional options given to the 
data model. Note that the EntingStrategy only supports one candidate, as 
each generated candidate is optimal (so generating multiple would generate 
duplicates).

In [2]:
from bofire.benchmarks.single import Himmelblau
import bofire.strategies.mapper as strategy_mapper

benchmark = Himmelblau()

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

c:\users\tobyb\phd\bofire\bofire\utils\cheminformatics.py:19: UserWarning: mordred not installed. Mordred molecular descriptors cannot be used.
  warnings.warn(
c:\users\tobyb\phd\bofire\bofire\surrogates\xgb.py:12: UserWarning: xgboost not installed, BoFire's `XGBoostSurrogate` cannot be used.
  warnings.warn("xgboost not installed, BoFire's `XGBoostSurrogate` cannot be used.")


In [3]:
from bofire.data_models.strategies.api import SoboStrategy
from bofire.data_models.acquisition_functions.api import qNEI

sobo_strategy_data_model = SoboStrategy(domain=benchmark.domain, acquisition_function=qNEI())
sobo_strategy = strategy_mapper.map(sobo_strategy_data_model)

sobo_strategy.tell(experiments=experiments)
sobo_strategy.ask(candidate_count=2)


,x_1,x_2,y_pred,y_sd,y_des
0,-0.101251,0.459799,19.605878,329.566771,-19.605878
1,-6.000000,-3.201815,28.583902,237.872201,-28.583902


In [4]:
from bofire.data_models.strategies.api import EntingStrategy

enting_params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
solver_params = {"solver_name": "gurobi"}

enting_strategy_data_model = EntingStrategy(domain=benchmark.domain, enting_params=enting_params, solver_params=solver_params,
                                            learn_from_candidates_coeff=10.0)
enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=2)

,x_1,x_2,y_pred,y_sd,y_des
0,-6.0000,-6.000000,66.126943,0.370573,-66.126943
0,-4.7772,-5.574912,70.899094,0.262034,-70.899094


When `candidate_count > 1`, the strategy will fit to each candidate in order to produce the next candidate. This means that sequential calls to `EntingStrategy.ask()` will tend to be further from the ground truth.

In [5]:
enting_strategy.ask(candidate_count=5)

,x_1,x_2,y_pred,y_sd,y_des
0,-6.000000,-3.375590,-27.149744,0.321733,27.149744
0,-3.455760,-1.769781,178.232196,0.380717,-178.232196
0,-4.859750,-2.469790,-0.359267,0.291980,0.359267
0,-4.640611,-3.375569,-48.866791,0.216498,48.866791
0,-2.972560,-3.272680,157.676060,0.271642,-157.676060


## Multi Objective BO

In [6]:
from bofire.benchmarks.multi import DTLZ2
import bofire.strategies.mapper as strategy_mapper

benchmark = DTLZ2(dim=2, num_objectives=2)

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

In [7]:
enting_params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
solver_params = {"solver_name": "gurobi"}

enting_strategy_data_model = EntingStrategy(domain=benchmark.domain, enting_params=enting_params, solver_params=solver_params)
enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=1)

,x_0,x_1,f_0_pred,f_1_pred,f_0_sd,f_1_sd,f_0_des,f_1_des
0,1.0,1.0,0.201118,0.736655,0.490467,0.490467,-0.201118,-0.736655


## Maximize single objective

In [8]:
from bofire.benchmarks.single import Ackley

benchmark = Ackley()

samples = benchmark.domain.inputs.sample(10)
experiments = benchmark.f(samples, return_complete=True)

In [9]:
enting_params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
solver_params = {"solver_name": "gurobi"}

enting_strategy_data_model = EntingStrategy(domain=benchmark.domain, enting_params=enting_params, solver_params=solver_params)
enting_strategy = strategy_mapper.map(enting_strategy_data_model)

enting_strategy.tell(experiments=experiments)
enting_strategy.ask(candidate_count=1)

,x_1,x_2,y_pred,y_sd,y_des
0,-19.223244,32.768,19.7255,0.525245,19.7255


## Compare domain to problem config

BoFire defines its problems using Domains, whereas ENTMOOT uses ProblemConfigs. 
This demonstrates the conversion from Domain to ProblemConfig, using the multi 
objective categorical problem defined in the ENTMOOT benchmark.

In [10]:
from bofire.data_models.domain.api import Domain, Inputs, Outputs
from bofire.data_models.features.api import (
    CategoricalInput,
    ContinuousInput,
    DiscreteInput,
    ContinuousOutput,
)
from bofire.data_models.objectives.api import MinimizeObjective


def build_multi_obj_categorical_problem(n_obj: int = 2, no_cat=False) -> Domain:
    """
    Builds a small test example which is used in Entmoot tests.
    """

    cat_feat = (
        []
        if no_cat
        else [CategoricalInput(key="x0", categories=("blue", "orange", "gray"))]
    )
    input_features = Inputs(
        features=cat_feat
        + [
            DiscreteInput(key="x1", values=[5, 6]),
            DiscreteInput(key="x2", values=[0, 1]),  # binary
            ContinuousInput(key="x3", bounds=[5.0, 6.0]),
            ContinuousInput(key="x4", bounds=[4.6, 6.0]),
            ContinuousInput(key="x5", bounds=[5.0, 6.0]),
        ]
    )

    output_features = Outputs(
        features=[
            ContinuousOutput(
                key=f"y{i}", objective=MinimizeObjective(w=1.0, bounds=[0.0, 1.0])
            )
            for i in range(n_obj)
        ]
    )

    domain = Domain(inputs=input_features, outputs=output_features)

    return domain


In [11]:
from bofire.utils.entmoot import domain_to_problem_config
import entmoot.benchmarks as ent
from entmoot.problem_config import ProblemConfig

domain = build_multi_obj_categorical_problem()
problem_config, _ = domain_to_problem_config(domain)
problem_config_ent = ProblemConfig()
ent.build_multi_obj_categorical_problem(problem_config_ent)

print(problem_config, "\n", problem_config_ent)


PROBLEM SUMMARY
---------------
features:
x0 :: Categorical :: ('blue', 'orange', 'gray') 
x1 :: Integer :: (5, 6) 
x2 :: Binary :: (0, 1) 
x3 :: Real :: (5.0, 6.0) 
x4 :: Real :: (4.6, 6.0) 
x5 :: Real :: (5.0, 6.0) 

objectives:
y0 :: MinObjective
y1 :: MinObjective 
 
PROBLEM SUMMARY
---------------
features:
feat_0 :: Categorical :: ('blue', 'orange', 'gray') 
feat_1 :: Integer :: (5, 6) 
feat_2 :: Binary :: (0, 1) 
feat_3 :: Real :: (5.0, 6.0) 
feat_4 :: Real :: (4.6, 6.0) 
feat_5 :: Real :: (5.0, 6.0) 

objectives:
obj_0 :: MinObjective
obj_1 :: MinObjective
